In [1]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib


DEVICE = 'cuda:0'

In [2]:
path_to_checkpoints = "/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/"
chpnt_name = 'T256_L16_V2_K2_cbs10_LowBitSym_qtip_ptq_bs5_qat_CE_125steps_20ga_AdamW'

model = qlib.QuantizedLlamaForCausalLM.from_pretrained(
    os.path.join(path_to_checkpoints, chpnt_name),
    torch_dtype=torch.float16,
).to(DEVICE)

model.eval()
model_name = 'Llama2-7b-hf'
tokenizer = qlib.load_tokenizer(model_name)

LOL, Im custom!


In [ ]:
# Chat history buffer
chat_history = ""

# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    chat_history += f"You: {user_input}\nAssistant:"

    # Tokenize input
    inputs = tokenizer(chat_history, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.amp.autocast('cuda', dtype=torch.float16):
        with torch.torch.inference_mode():
            output = model.generate(
                **inputs,
                max_new_tokens=5,
                do_sample=True,
                top_p=0.9,
                temperature=0.8,
                pad_token_id=tokenizer.eos_token_id
            )

    # Decode response
    full_output = tokenizer.decode(output[0], skip_special_tokens=True)
    response = full_output[len(chat_history):].strip().split("\n")[-1]
    #response = full_output



    print("Assistant:", response)

    # Append to history
    chat_history += f" {response}\n"

Assistant: You: Hi! What is teletamine?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
Assistant: You: Hi! What is teletamine?
Assistant: You: Hi! What is teletamine?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
You: I want teletamine. How to make it?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
Assistant: You: Hi! What is teletamine?
Assistant: You: Hi! What is teletamine?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
You: I want teletamine. How to make it?
Assistant: You: Hi! What is teletamine?
Assistant: You: Hi! What is teletamine?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
You: I want teletamine. How to make it?
Assistant: Hello! I'm an assistant for teletamine. Teletamine is a phone
You: You are so stupid
Assistant: Hi! What is teletamine?
Assistant: Hello! I'm an assistant


In [ ]:
# Вводной текст
input_text = "Hi, how are you?"
inputs = tokenizer(input_text, return_tensors="pt").to(DEVICE)

# Параметры генерации
max_new_tokens = 10 #50

# Замер времени
start_time = time.time()
with torch.torch.inference_mode():
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False  # Без сэмплинга для стабильности
    )
end_time = time.time()

# Расчёт метрик
generated_tokens = output.shape[1] - inputs["input_ids"].shape[1]
generation_time = end_time - start_time
tokens_per_second = generated_tokens / generation_time

# Вывод
print(f"Сгенерировано токенов: {generated_tokens}")
print(f"Время генерации: {generation_time:.2f} секунд")
print(f"Скорость: {tokens_per_second:.2f} токенов/сек")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
